<a href="https://githubtocolab.com/Astolfo2332/HyM_prediction/blob/main/0.2_Limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Limpieza de datos

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
'chmod 600 /root/.kaggle/kaggle.json' #Para proteger la información

In [ ]:
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f articles.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f customers.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f transactions_train.csv
! mkdir Data
! unzip transactions_train.csv.zip -d Data
! unzip customers.csv.zip -d Data
! unzip articles.csv.zip -d Data


## 1. Generación de datos faltantes

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import preprocessing as ps
from difflib import SequenceMatcher
import csv

En este caso vamos a generar 5% de datos faltantes en las columnas de interés, así que aleatoriamente se desorganizan los datasets antes de unirlos en uno y se eliminaran en parte de transacciones el 5% de artículos comprados, en clientes; la edad y finalmente en artículos; el tipo de articulo.

In [38]:
client=pd.read_csv("Data/customers.csv")

In [39]:
sells=pd.read_csv("Data/transactions_train.csv")

In [40]:
articles=pd.read_csv("Data/articles.csv")

In [41]:
def makeitrandom(client,sells,articles):
    client=client.sample(frac=1) #Desorganizamos las filas
    sells=sells.sample(frac=1)
    articles=articles.sample(frac=1)
    client=client.reset_index(drop=True) #Reiniciamos los index
    sells=sells.reset_index(drop=True)
    articles=articles.reset_index(drop=True)
    return client,sells,articles

In [42]:
client,sells,articles=makeitrandom(client,sells,articles)

In [43]:
f=0.05
n_client=int(len(client)*f)
n_sells=int(len(sells)*f)
n_articles=int(len(articles)*f)

In [44]:
client.loc[:n_client,"age"]=np.nan
sells.loc[:n_sells,"article_id"]=np.nan
articles.loc[:n_articles,"product_type_name"]=np.nan

In [45]:
client,sells,articles=makeitrandom(client,sells,articles)

In [46]:
def faltap(data):
    total      = data.isnull().sum().sort_values(ascending=False)
    percent    = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending=False)
    missing  = pd.concat([total,percent],axis=1,keys=["Total","Percent"])
    return missing

In [47]:
faltap(client)

,Total,Percent
Active,907576,66.150819
FN,895050,65.237831
age,83691,6.100016
fashion_news_frequency,16009,1.166854
club_member_status,6062,0.441843
customer_id,0,0.000000
postal_code,0,0.000000


In [48]:
faltap(sells)

,Total,Percent
article_id,1589417,5.000003
t_dat,0,0.000000
customer_id,0,0.000000
price,0,0.000000
sales_channel_id,0,0.000000


In [49]:
faltap(articles)

,Total,Percent
product_type_name,5278,5.000853
detail_desc,416,0.394156
perceived_colour_master_name,0,0.000000
garment_group_name,0,0.000000
garment_group_no,0,0.000000
section_name,0,0.000000
section_no,0,0.000000
index_group_name,0,0.000000
index_group_no,0,0.000000
index_name,0,0.000000


## 2. agrupación de datos y eliminación de variables

En este caso vamos a proceder a primero hacer una correlación con las compras y los artículos de cada cliente, para generar un dataframe, que nos resuma los artículos comprados Por cliente, el dataset al ser tan grande primero se le hará una reducción a las transacciones y una división entre train y test por fechas

In [50]:
a=len(sells)//2
sells=sells.loc[0:a]
sells.index=sells.customer_id.values
del(sells["sales_channel_id"])
sells.head()

,t_dat,customer_id,article_id,price
59e8d7241a8ab4f6800a7ba33889cf7208fbf942a23de87bbae40a683c4d1ca2,2019-03-10,59e8d7241a8ab4f6800a7ba33889cf7208fbf942a23de8...,669713001.0,0.015237
7d04ee4d5c83ddd26966f268625021c4e856c04d2721b256599b2ee57638fb17,2020-05-26,7d04ee4d5c83ddd26966f268625021c4e856c04d2721b2...,893796002.0,0.015237
0180d0ef5499cd4dfd920c19821c5f1e3f6692662e7d7840925245875012ceb3,2018-10-04,0180d0ef5499cd4dfd920c19821c5f1e3f6692662e7d78...,667499001.0,0.021593
43aefe1821b101a7954873e52b9beb961abe099fb240c8ad20b5d774fc779773,2020-06-18,43aefe1821b101a7954873e52b9beb961abe099fb240c8...,879781002.0,0.016932
9a5dcc5c8798b5a6ede2d1accd7f8ed1303a3acd69837b00d21283ac4e3b5ba1,2020-03-06,9a5dcc5c8798b5a6ede2d1accd7f8ed1303a3acd69837b...,829646005.0,0.042356


In [51]:
client.index=client.customer_id.values
del(client["postal_code"])
del(client["customer_id"])
client.head()

,FN,Active,club_member_status,fashion_news_frequency,age
1c5d162edba35975989eeeaeaabae56b9ce75c5b6e6132d645538ab251c99fb9,1.0,1.0,ACTIVE,Regularly,46.0
9d4bbc41e2e06f00b2f5ef728de87123164548fe38ec4b7ade533f818cc23523,NaN,NaN,ACTIVE,NONE,59.0
0486a01ab014043171ca1a5a395a26da23466c14628b3df81c57fb2073d76755,NaN,NaN,ACTIVE,NONE,30.0
e9da25c06e1abca25a39f35b97de3f925afc63211d1c748397fa2ff8309686be,NaN,NaN,PRE-CREATE,NONE,32.0
ef4da7fea78017551b958f9fb7a664d016b4b93d27bb0e879c5f4fd64d54e3d4,NaN,NaN,PRE-CREATE,NONE,70.0


In [52]:
sells=sells.join(client)
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001.0,0.044051,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2019-11-22,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785186005.0,0.016932,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2019-05-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006.0,0.050831,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,887593002.0,0.021593,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785710001.0,0.024407,NaN,NaN,ACTIVE,NONE,49.0


In [53]:
sells=sells.reset_index(drop=True)

In [54]:
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
0,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001.0,0.044051,NaN,NaN,ACTIVE,NONE,49.0
1,2019-11-22,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785186005.0,0.016932,NaN,NaN,ACTIVE,NONE,49.0
2,2019-05-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006.0,0.050831,NaN,NaN,ACTIVE,NONE,49.0
3,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,887593002.0,0.021593,NaN,NaN,ACTIVE,NONE,49.0
4,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785710001.0,0.024407,NaN,NaN,ACTIVE,NONE,49.0


## 2.1 Relleno de datos vacíos  

En este caso vamos a reemplazar los datos faltantes, en el caso de el articulo, fashion_news, con la moda, el FN y active con 0. Y age con el promedio.

In [55]:
sells["article_id"].fillna(sells.article_id.mode()[0], inplace=True)
sells["FN"].fillna(0, inplace=True)
sells["Active"].fillna(0, inplace=True)
sells["club_member_status"].fillna(sells.club_member_status.mode()[0], inplace=True)
sells["fashion_news_frequency"].fillna(sells.fashion_news_frequency.mode()[0], inplace=True)
sells["age"].fillna(int(sells.age.mean()), inplace=True)

In [56]:
sells.isnull().sum()

t_dat                     0
customer_id               0
article_id                0
price                     0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
dtype: int64

## 2.2 Filtrado de datos y concatenación final

Ahora vamos a filtrar los artículos para las columnas que consideramos afecten más la compra futura, como es el tipo de articulo, color, la sección a la que pertenece (ya que de ahí podríamos inferir el genero del cliente)

In [57]:
sells.index=sells.article_id.values
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
625548001.0,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001.0,0.044051,0.0,0.0,ACTIVE,NONE,49.0
785186005.0,2019-11-22,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785186005.0,0.016932,0.0,0.0,ACTIVE,NONE,49.0
568601006.0,2019-05-25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006.0,0.050831,0.0,0.0,ACTIVE,NONE,49.0
887593002.0,2020-03-21,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,887593002.0,0.021593,0.0,0.0,ACTIVE,NONE,49.0
785710001.0,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785710001.0,0.024407,0.0,0.0,ACTIVE,NONE,49.0


In [58]:
articles=articles[["product_type_name","article_id","colour_group_name","index_group_name","section_name"]]
articles.index=articles.article_id.values
del(articles["article_id"])

In [59]:
sells=sells.join(articles)
sells=sells.reset_index(drop=True)
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
0,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
1,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2,2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
3,2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,108775015.0,0.006763,1.0,1.0,ACTIVE,Regularly,50.0,Vest top,Black,Ladieswear,Womens Everyday Basics
4,2018-12-06,002323971cbd38fad4512d5114676e5e17eb262db02320...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,28.0,Vest top,Black,Ladieswear,Womens Everyday Basics


In [60]:
sells.index=pd.to_datetime(sells.t_dat)
del(sells["t_dat"])
sells.head()

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
t_dat,,,,,,,,,,,,
2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-07-14,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,108775015.0,0.006763,1.0,1.0,ACTIVE,Regularly,50.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2018-12-06,002323971cbd38fad4512d5114676e5e17eb262db02320...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,28.0,Vest top,Black,Ladieswear,Womens Everyday Basics


In [61]:
sells.tail()

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
t_dat,,,,,,,,,,,,
2020-09-16,98b6235b68deffd24b91233fc692549ba104724c8b1263...,956217002.0,0.059305,1.0,1.0,ACTIVE,Regularly,60.0,Dress,Black,Ladieswear,Womens Trend
2020-09-10,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,31.0,Dress,Black,Ladieswear,Womens Trend
2020-09-20,c89698e2626f3d98d8496e673c89ece5a8ee92914ee92d...,956217002.0,0.059305,1.0,1.0,ACTIVE,Regularly,53.0,Dress,Black,Ladieswear,Womens Trend
2020-09-14,cedd533807afaef98ca626a771d8c14986de84bdb43743...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,57.0,Dress,Black,Ladieswear,Womens Trend
2020-09-12,d51d1c7b07110ba2571d22849c1fdb7e61d7cbc1d58ada...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,26.0,Dress,Black,Ladieswear,Womens Trend


## 3 Transformación de variables categóricas a numéricas

In [62]:
var_categoricas = ['customer_id', 'club_member_status',"fashion_news_frequency","product_type_name","colour_group_name","index_group_name","section_name"]
encoder = ps.LabelEncoder()

for i in var_categoricas:
    
    sells[i] = encoder.fit_transform(sells[i])

sells.article_id=sells.article_id.astype(int)
sells["FN"]=sells["FN"].astype(int)
sells.Active=sells.Active.astype(int)
sells.age=sells.age.astype(int)
sells=sells.reset_index(drop=True)
print (sells.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15894163 entries, 0 to 15894162
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   customer_id             int32  
 1   article_id              int32  
 2   price                   float64
 3   FN                      int32  
 4   Active                  int32  
 5   club_member_status      int32  
 6   fashion_news_frequency  int32  
 7   age                     int32  
 8   product_type_name       int32  
 9   colour_group_name       int32  
 10  index_group_name        int32  
 11  section_name            int32  
dtypes: float64(1), int32(11)
memory usage: 788.2 MB
None


Separamos directamente los datos de test y de train

In [63]:
sells

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
0,286,108775015,0.008458,0,0,0,1,48,119,1,2,43
1,286,108775015,0.008458,0,0,0,1,48,119,1,2,43
2,286,108775015,0.008458,0,0,0,1,48,119,1,2,43
3,518,108775015,0.006763,1,1,0,3,50,119,1,2,43
4,672,108775015,0.008458,1,1,0,3,28,119,1,2,43
...,...,...,...,...,...,...,...,...,...,...,...,...
15894158,741829,956217002,0.059305,1,1,0,3,60,31,1,2,53
15894159,924005,956217002,0.059305,0,0,0,1,31,31,1,2,53
15894160,974829,956217002,0.059305,1,1,0,3,53,31,1,2,53
15894161,1005272,956217002,0.059305,0,0,0,1,57,31,1,2,53


Para simplificar el problema vamos a cambiar el enfoque del problema, ahora vamos a intentar predecir la recompra de los artículos

In [64]:
sell_sorted = sells.sort_values(['customer_id', 'article_id'])
sells['rebuy'] = sell_sorted['article_id'].duplicated(keep='first')
sells['rebuy'] = sells['rebuy'].astype(int)


In [65]:
sells

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name,rebuy
0,286,108775015,0.008458,0,0,0,1,48,119,1,2,43,0
1,286,108775015,0.008458,0,0,0,1,48,119,1,2,43,1
2,286,108775015,0.008458,0,0,0,1,48,119,1,2,43,1
3,518,108775015,0.006763,1,1,0,3,50,119,1,2,43,1
4,672,108775015,0.008458,1,1,0,3,28,119,1,2,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15894158,741829,956217002,0.059305,1,1,0,3,60,31,1,2,53,1
15894159,924005,956217002,0.059305,0,0,0,1,31,31,1,2,53,1
15894160,974829,956217002,0.059305,1,1,0,3,53,31,1,2,53,1
15894161,1005272,956217002,0.059305,0,0,0,1,57,31,1,2,53,1


In [66]:
sells.to_csv("data_set.csv",index=False)